In [20]:
import pandas as pd
import numpy as np
from bitfile import *
from constants import *

In [ ]:
ENC_FILE_PATH = "/Users/arroyoruizj/Desktop/comps_f25_rgj/dnazip/data/output/HG004_GRCh38_Encoded.bin"

In [22]:
TWO_BIT_ENCODING = {
    "00": "A",
    "01": "C",
    "10": "G",
    "11": "T",
}

In [23]:
def add_padding(bit_string):

    # Find first VINT
    padding = 0
    while bit_string[padding] != '1':

        padding += 1

    return bit_string[padding:]





In [24]:
def readBinFile(file_to_bin_file):

    with open(file_to_bin_file, 'rb') as f:

        binary_data = f.read()

        bit_string = BytesToBitString(binary_data)

    return bit_string



In [25]:
def parse_vints(bit_string, size):
    items = []
    shifted_bits = 0

    for _ in range(size):
        pos, bits_shifted = readBitVINT_from(bit_string, shifted_bits)
        shifted_bits += bits_shifted
        items.append(pos)

    return items, shifted_bits

In [26]:
CHROMOSOMES = ['chr1']

In [ ]:
def decode(bit_string):
        
        for chr in CHROMOSOMES:

            bit_string = add_padding(bit_string)

            bitmap_size, bits_shifted = readBitVINT(bit_string)
            bit_string = bit_string[bits_shifted:]

            bitmap = bit_string[:bitmap_size]
            bit_string = bit_string[bitmap_size:]

            # print("Bitmap Size: " + str(bitmap_size))

            snp_size, bits_shifted = readBitVINT(bit_string)
            bit_string = bit_string[bits_shifted:]

            # print("SNPs Size: " + str(snp_size))
            
            snp_pos, snp_pos_bits = parse_vints(bit_string, snp_size)
            bit_string = bit_string[snp_pos_bits:]

            # print("SNP Positions Decoded")

            snp_nucs = []

            for i in range(0, snp_size * 2, 2):

                two_bits = bit_string[i:i+2]
                nuc = TWO_BIT_ENCODING[two_bits]
                snp_nucs.append(nuc)

            # print("SNP Nucleotides Decoded")


            bit_string = bit_string[snp_size * 2:]

            bit_string = add_padding(bit_string)
        
            del_size, bits_shifted = readBitVINT(bit_string)
            bit_string = bit_string[bits_shifted:]

            # print("Dels Size: " + str(del_size))

            del_pos, del_pos_bits = parse_vints(bit_string, del_size)
            bit_string = bit_string[del_pos_bits:]

            # print("Deletion Positions Decoded")

            del_size, del_size_bits = parse_vints(bit_string, del_size)
            bit_string = bit_string[del_size_bits:]

            # print("Deletion Sizes Decoded")

            bit_string = add_padding(bit_string)

            ins_size, bits_shifted = readBitVINT(bit_string)
            bit_string = bit_string[bits_shifted:]

            # print("Ins Size: " + str(ins_size))

            ins_pos, ins_pos_bits = parse_vints(bit_string, ins_size)
            bit_string = bit_string[ins_pos_bits:]

            # print("Insertion Positions Decoded")

            ins_lens, ins_len_bits = parse_vints(bit_string, ins_size)
            bit_string = bit_string[ins_len_bits:]

            bitstr_len, bits_shifted = readBitVINT(bit_string)
            bit_string = bit_string[bits_shifted:]

            # print("Bitstr Len: " + str(bitstr_len))

            bitmap = bit_string[:bitstr_len]
            bit_string = bit_string[bitstr_len:]

            #Making df example:

            snp_data = {
                 "snp_pos":None,
                 "snp_nucs":None
            }

            snp_data["snp_pos"] = snp_pos
            snp_data["snp_nucs"] = snp_nucs
            
            snp_df = pd.DataFrame(snp_data)
            display(snp_df)



In [28]:
bit_string = readBinFile(ENC_FILE_PATH)


In [29]:
decode(bit_string)

,snp_pos,snp_nucs
0,10000458,C
1,100015152,T
2,10001749,A
3,10002030,A
4,10002110,A
...,...,...
273134,99962686,T
273135,99966334,T
273136,9997612,A
273137,9997877,C
